In [1]:
#loading the packages
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer 
import nltk 
from nltk.tokenize import word_tokenize 
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
#creating dummy data with 2 columns Text, and Label

df_data = pd.DataFrame([{'Text': 'reading assignment', 'Label':'Ham'}, 
                        {'Text': 'analysis and design assignment', 'Label':'Ham'},
                        {'Text': 'hello how are you', 'Label':'Ham'},
                        {'Text': 'requirements analysis', 'Label':'Ham'}, 
                        {'Text': 'win free', 'Label':'Spam'},
                        {'Text': 'lotto millions', 'Label':'Spam'},
                        {'Text': 'win win win millions', 'Label':'Spam'},
                        {'Text': 'free offers', 'Label':'Spam'}]) 

df_data

,Text,Label
0,reading assignment,Ham
1,analysis and design assignment,Ham
2,hello how are you,Ham
3,requirements analysis,Ham
4,win free,Spam
5,lotto millions,Spam
6,win win win millions,Spam
7,free offers,Spam


In [3]:
#pandas filter label column with only ham
df_data_ham = df_data[df_data.Label == 'Ham']
df_data_ham

,Text,Label
0,reading assignment,Ham
1,analysis and design assignment,Ham
2,hello how are you,Ham
3,requirements analysis,Ham


In [4]:
#panda filtering label column with spam
df_data_spam = df_data[df_data.Label == 'Spam']
df_data_spam

,Text,Label
4,win free,Spam
5,lotto millions,Spam
6,win win win millions,Spam
7,free offers,Spam


##Question 2 - Vectorize the text to create a Term Document Frequency matrix.


In [5]:
#tdf for all text
#vecotirizng is converting text to numeric repreentation, often shown as a matrix

vec = CountVectorizer() 
tdf = vec.fit_transform(df_data['Text']) 
total_words = len(vec.get_feature_names())

In [6]:
print(tdf.toarray()), print(vec.get_feature_names())

[[0 0 0 1 0 0 0 0 0 0 0 1 0 0 0]
 [1 1 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 1 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 3 0]
 [0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]]
['analysis', 'and', 'are', 'assignment', 'design', 'free', 'hello', 'how', 'lotto', 'millions', 'offers', 'reading', 'requirements', 'win', 'you']


(None, None)

In [7]:
#TDM for ham

count_vect_ham = CountVectorizer() #intializing Vectorizier

#creating our term document matrix
tdf_ham = count_vect_ham.fit_transform(df_data_ham['Text']) 
tdm_ham = pd.DataFrame(tdf_ham.toarray(), columns=count_vect_ham.get_feature_names())
word_list_ham = count_vect_ham.get_feature_names()

#dictionary that gives a word count, dictionary is a data structure
tdm_ham_dict = dict(zip(word_list_ham, tdf_ham.toarray().sum(axis=0) ))
print(tdm_ham_dict)


{'analysis': 2, 'and': 1, 'are': 1, 'assignment': 2, 'design': 1, 'hello': 1, 'how': 1, 'reading': 1, 'requirements': 1, 'you': 1}


In [75]:
tdm_ham

,analysis,and,are,assignment,design,hello,how,reading,requirements,you
0,0,0,0,1,0,0,0,1,0,0
1,1,1,0,1,1,0,0,0,0,0
2,0,0,1,0,0,1,1,0,0,1
3,1,0,0,0,0,0,0,0,1,0


In [52]:
df_data_ham

,Text,Label
0,reading assignment,Ham
1,analysis and design assignment,Ham
2,hello how are you,Ham
3,requirements analysis,Ham


In [51]:
tdm_ham

,analysis,and,are,assignment,design,hello,how,reading,requirements,you
0,0,0,0,1,0,0,0,1,0,0
1,1,1,0,1,1,0,0,0,0,0
2,0,0,1,0,0,1,1,0,0,1
3,1,0,0,0,0,0,0,0,1,0


In [8]:
#TDM for spam

#creating our term document matrix
count_vect_spam = CountVectorizer()
tdf_spam = count_vect_spam.fit_transform(df_data_spam['Text'])
tdm_spam = pd.DataFrame(tdf_spam.toarray(), columns=count_vect_spam.get_feature_names())
word_list_spam = count_vect_spam.get_feature_names()
tdm_spam_dict = dict(zip(word_list_spam, tdf_spam.toarray().sum(axis=0) ))
print(tdm_spam_dict)

{'free': 2, 'lotto': 1, 'millions': 2, 'offers': 1, 'win': 4}


##Question 3 - Develop a Naïve Bayes classifier to detect Spam.


In [9]:
#creating prior probability of spam, and ham
prior_prob_spam = df_data_spam.shape[0] / df_data.shape[0] 
prior_prob_ham = df_data_ham.shape[0] / df_data.shape[0]

In [71]:
#creating a UDF to calculate bayes posterior prob
def posterior_prob(test_sentence): 
  
    prob_spam_given_all_words = 1 
    prob_ham_given_all_words = 1 
    ham_words_count = tdf_ham.toarray().sum() #word count of ham 
    spam_words_count = tdf_spam.toarray().sum() #word count of spam

    words = word_tokenize(test_sentence) #create  a list of words
    for word in words: # P(Word | Spam) with Laplace smoothing 

        #P(Words | Spam) w/ Laplace smoothing (alpha = 1)
        prob_spam_given_word = ((tdm_spam_dict.get(word, 0) + 1)/(spam_words_count + total_words))
        prob_spam_given_all_words = prob_spam_given_all_words * prob_spam_given_word

        #P(Words | Ham) w/ Laplace smoothing (alpha = 1)
        prob_ham_given_word = ((tdm_ham_dict.get(word, 0) + 1)/(ham_words_count + total_words))
        prob_ham_given_all_words = prob_ham_given_all_words * prob_ham_given_word

        #P(Ham | words) =  P(Ham) * P(Words | Ham)
        posterior_ham = prior_prob_ham * prob_ham_given_all_words

        # P(Spam | words) = P(Spam) * P(Words | Spam)
        posterior_spam = prior_prob_spam * prob_spam_given_all_words

        #compare posterior of Ham vs. Posterior of Spam
        CLASS = 'Ham' if posterior_ham > posterior_spam else 'Spam'

        #P(H) , P(S)
        prob_ham = posterior_ham / (posterior_spam + posterior_ham) 
        prob_spam = posterior_spam / (posterior_spam + posterior_ham) 
  
    return (prob_ham, prob_spam, CLASS)


In [ ]:
['free', 'deal']

In [72]:
prob_ham, prob_spam, label = posterior_prob('free deal')
print(prob_ham, prob_spam, label)

0.22226173541963015 0.7777382645803698 Spam
